In [1]:
# Install Python 3.9 and build tools
!apt-get update
!apt-get install -y python3.9 python3.9-dev python3.9-distutils build-essential libpython3.9-dev

# Install pip 23.2.1 and build tools
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.9 get-pip.py
!python3.9 -m pip install pip==23.2.1 setuptools==65.6.3 wheel==0.38.4
!python3.9 -m pip --version

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [1]:
# Install core dependencies
!python3.9 -m pip install numpy==1.23.0 protobuf==3.20.0 tensorflow==2.8.0 pandas==1.5.3 matplotlib==3.4.3 matplotlib-inline==0.1.6 ipython==7.34.0 tensorflowjs==3.18.0 lxml==4.9.1 PyYAML==6.0 gin-config==0.5.0 absl-py==1.4.0 pycocotools==2.0.6 six==1.16.0 tensorflow-model-optimization==0.7.3 Cython==0.29.36 fire==0.4.0 librosa==0.8.1 neural-structured-learning==1.4.0 numba==0.53.1 scann==1.2.6 sentencepiece==0.1.99 tensorflow-addons==0.17.1 tensorflow-datasets==4.9.2 tf-models-official==2.3.0 urllib3==1.25.11

# Install tflite packages
!python3.9 -m pip install tflite-support==0.4.3
!python3.9 -m pip install tflite-model-maker==0.4.2

# Verify installations
!python3.9 -m pip list | grep -E "numpy|protobuf|tensorflow|tflite|pandas|matplotlib|ipython|tensorflowjs|lxml|PyYAML|gin-config|absl-py|pycocotools|six|tensorflow-model-optimization|Cython|fire|librosa|neural-structured-learning|numba|scann|sentencepiece|tensorflow-addons|tensorflow-datasets|tf-models-official|urllib3|sounddevice|pybind11|CFFI|pycparser|setuptools|wheel"

  Obtaining dependency information for Cython==0.29.36 from https://files.pythonhosted.org/packages/35/cb/24999b0f2774ddc498d8e885f0b5f0441e008e1c100b7810498913b5bf55/Cython-0.29.36-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata
  Using cached Cython-0.29.36-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
  Using cached fire-0.4.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for librosa==0.8.1 from https://files.pythonhosted.org/packages/54/19/a0e2bdc94bc0d1555e4f9bc4099a0751da83fa6e1e6157ec005564f8a98a/librosa-0.8.1-py3-none-any.whl.metadata
  Using cached librosa-0.8.1-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for neural-structured-learning==1.4.0 from https://files.pythonhosted.org/packages/55/14/f350952528cd69b47ca9f908ac153b841361824fec4e787830d8a9834100/neural_structured_learning-1.4.0-py2.py3-none-any.whl.metadata
  U

In [2]:
# Enable GPU (optional, do manually: Runtime > Change runtime type > GPU)

# Write the training script
with open('train.py', 'w') as f:
    f.write('''
import os
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

print("Starting script...")

dataset_path = '/content/soil_dataset'
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset folder {dataset_path} not found.")
print(f"Dataset path exists: {dataset_path}")

# List classes
classes = os.listdir(dataset_path)
print(f"Classes found: {classes}")

# Load dataset
print("Loading dataset...")
data = DataLoader.from_folder(dataset_path)
print(f"Dataset loaded successfully, total images: {len(data)}")

# Split dataset
print("Splitting dataset...")
train_data, test_data = data.split(0.8)
print(f"Dataset split: {len(train_data)} train, {len(test_data)} test")

# Create and train model
print("Training model...")
model = image_classifier.create(
    train_data,
    model_spec='efficientnet_lite0',
    validation_data=test_data,
    epochs=10,
    batch_size=8,  # Reduced to reduce memory usage
    shuffle=True
)
print("Model training completed")

# Evaluate model
print("Evaluating model...")
loss, accuracy = model.evaluate(test_data)
print(f"Test accuracy: {accuracy*100:.2f}%")

# Export model
print("Exporting model...")
model.export(export_dir='/content/')
print("Model exported to /content/model.tflite")
''')

# Run the script
!python3.9 train.py

2025-04-24 11:50:02.286130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-24 11:50:02.286224: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.9/dist-pa

In [4]:
import os

# Path to your dataset
dataset_path = '/content/soil_dataset'

# Get class names (sorted to ensure consistent order)
class_names = sorted(os.listdir(dataset_path))

# Write class names to labels.txt
with open('/content/labels.txt', 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

# Verify the file
!cat /content/labels.txt

Alluvial
Black
Clay
Not Soil
Red


In [5]:
# Check if model exists
!ls /content/model.tflite /content/labels.txt

# Download files
from google.colab import files
files.download('/content/model.tflite')
files.download('/content/labels.txt')  # If generated

/content/labels.txt  /content/model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import tensorflow as tf

# Load the model
interpreter = tf.lite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'input_1', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'Identity', 'index': 168, 'shape': array([1, 5], dtype=int32), 'shape_signature': array([-1,  5], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
